# Segmenting and clustering neighbourhoods in Toronto

All the parts of the specific problem are shown in the same notebook. I designated each problem with appropriate markdowns.

## Part 1 : Scraping Wikipedia data to get the different Boroughs and neighborhoods 

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
tables = pd.read_html('List_of_postal_codes_of_Canada:_M.html', header=0) # reading the html file for tables
tables

In [ ]:
torontoData = tables[0] #extracting the required table 
torontoData

In [ ]:
torontoData.columns = ['PostalCode', 'Borough', 'Neighbourhood']
torontoData = torontoData[torontoData['Borough']!='Not assigned'].reset_index(drop=True)

In [ ]:
torontoData

In [ ]:
#grouping neighborhoods based on duplicates in 'PostalCode' column
newcol = []
for posC in torontoData['PostalCode'] :
    l = torontoData[torontoData['PostalCode']==posC]['Neighbourhood'].values 
    newcol.append(', '.join(map(str, l)))
torontoData['Neighbourhood'] = newcol
torontoData = torontoData.drop_duplicates(subset = 'PostalCode', keep = "first").reset_index(drop=True)

In [ ]:
torontoData

In [ ]:
torontoData.shape[0]

## Part 2 : Getting the latitude and logitude of the neighborhoods

Geocoder takes forever to run on even one postal code. So I used the provided link to the csv file directly for this part.

In [ ]:
#import geocoder # import geocoder

#postal_code = 'M5G'
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [ ]:
geo_coord = pd.read_csv("http://cocl.us/Geospatial_data")
geo_coord.head()

In [ ]:
geo_coord.columns = ['PostalCode','Latitude','Longitude']
torontoData = torontoData.join(geo_coord.set_index('PostalCode'), on='PostalCode')
torontoData

## Part 3 :  Exploring venues and segmenting the neighborhoods

I am working here with venues associated with the borough - North York.

### Explore neighbourhoods in North York

In [ ]:
NorthYork_data = torontoData[torontoData['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

In [ ]:
address = 'North York, Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [ ]:
CLIENT_ID = 'CR2M0NQWJWLMJB5EQPBKSCQ3J3WGD1NUSZNW4QJNCRPGFUUV' # your Foursquare ID
CLIENT_SECRET = 'QH5B4UMVNBBPUCV3BWBNJ0XVFQCCXR2XT2CQV5YF0YMBAT4L' # your Foursquare Secret
VERSION = '20200226' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#defining a function to explore the nearby venues of each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighbourhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )

In [ ]:
print(NorthYork_venues.shape)
NorthYork_venues.head()

In [ ]:
NorthYork_venues.groupby('Neighbourhood').count() 

Finding number of unique venue categories

In [ ]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

##  Analyze Each Neighborhood

In [ ]:
# one hot encoding
NY_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighbourhood'] = NorthYork_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
NY_grouped = NY_onehot.groupby('Neighbourhood').mean().reset_index()
NY_grouped

#### Let's print each neighborhood along with the top 4 most common venues

In [ ]:
num_top_venues = 4

for hood in NY_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### now put this into a dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = NY_grouped['Neighbourhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Clustering Neighborhoods

In [ ]:
# set number of clusters
kclusters = 4

NY_grouped_clustering = NY_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NY_merged = NorthYork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

NY_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = cm.viridis(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighbourhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining clusters

In [ ]:
NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

In [ ]:
NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

In [ ]:
NY_merged.loc[NY_merged['Cluster Labels'] == 2, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

In [ ]:
NY_merged.loc[NY_merged['Cluster Labels'] == 3, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

#### Distinguishing features of the clusters

Clusters 1 and 3 have only one differentiating factor which also happens to be their respective 1st most common venue. So, Cluster 1 is associated with a proximity to baseball fields while 3 is more popular for its gym availability in the neighborhood. Cluster 0 seems to be defined by its numerous food and drinks places and Cluster 2 is appealing for its park venues mostly. 